## Data download

In [4]:
#XXX changes
!wget ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!wget ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

!wget ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3/plink_format/draft_2/relationships_w_pops_121708.txt

--2018-05-04 21:09:21--  ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotypes/hapmap3/plink_format/draft_2/hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
           => ‘hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 2607:f220:41e:250::11
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /hapmap/genotypes/hapmap3/plink_format/draft_2 ... done.
==> SIZE hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2 ... 10590722
==> PASV ... done.    ==> RETR hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2 ... done.
Length: 10590722 (10M) (unauthoritative)

hapmap3_r2_b36_fwd. 100%[===================>]  10.10M  12.7MB/s    in 0.8s    

2018-05-04 21:09:23 (12.7 MB/s) - ‘hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2’ saved [10590722]

--2018-05-04 21:09:23--  ftp://ftp.ncbi.nlm.nih.gov/hapmap/genotype

In [5]:
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.map.bz2
!bunzip2 hapmap3_r2_b36_fwd.consensus.qc.poly.ped.bz2

# Preparation

In [6]:
import os
from collections import defaultdict

## Loading HapMap meta-data

In [7]:
f = open('relationships_w_pops_121708.txt')
pop_ind = defaultdict(list)
f.readline()  # header
offspring = []
for l in f:
    toks = l.rstrip().split('\t')
    fam_id = toks[0]
    ind_id = toks[1]
    mom = toks[2]
    dad = toks[3]
    if mom != '0' or dad != '0':
        offspring.append((fam_id, ind_id))
    pop = toks[-1]
    pop_ind[pop].append((fam_id, ind_id))
f.close()

## Sub-sampling

In [8]:
os.system('plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap10 --thin 0.1 --geno 0.1')
os.system('plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap1 --thin 0.01 --geno 0.1')

0

## Getting only autosomal data

In [9]:
def get_non_auto_SNPs(map_file, exclude_file):
    f = open(map_file)
    w = open(exclude_file, 'w')
    for l in f:
        toks = l.rstrip().split('\t')
        chrom = int(toks[0])
        rs = toks[1]
        if chrom > 22:
            w.write('%s\n' % rs)
    w.close()

In [10]:
get_non_auto_SNPs('hapmap10.map', 'exclude10.txt')
get_non_auto_SNPs('hapmap1.map', 'exclude1.txt')
#get_non_auto_SNPs('hapmap3_r2_b36_fwd.consensus.qc.poly.map', 'exclude.txt')

In [11]:
!plink --recode --file hapmap10 --noweb --out hapmap10_auto --exclude exclude10.txt
!plink --recode --file hapmap1 --noweb --out hapmap1_auto --exclude exclude1.txt
#geno!!!
#!plink --recode --file hapmap3_r2_b36_fwd.consensus.qc.poly --noweb --out hapmap_auto --exclude exclude.txt

PLINK v1.90b4 64-bit (20 Mar 2017)             www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto.log.
Options in effect:
  --exclude exclude10.txt
  --file hapmap10
  --noweb
  --out hapmap10_auto
  --recode

Note: --noweb has no effect since no web check is implemented yet.
48283 MB RAM detected; reserving 24141 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (143840 variants, 1184 people).
--file: hapmap10_auto-temporary.bed + hapmap10_auto-temporary.bim +
hapmap10_auto-temporary.fam written.
143840 variants loaded from .bim file.
1184 people (589 males, 595 females) loaded from .fam.
--exclude: 138746 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 196 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333435363738394041

## Removing offspring

In [12]:
!plink --file hapmap10_auto --filter-founders --recode --out hapmap10_auto_noofs

PLINK v1.90b4 64-bit (20 Mar 2017)             www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto_noofs.log.
Options in effect:
  --file hapmap10_auto
  --filter-founders
  --out hapmap10_auto_noofs
  --recode

48283 MB RAM detected; reserving 24141 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (138746 variants, 1184 people).
--file: hapmap10_auto_noofs-temporary.bed + hapmap10_auto_noofs-temporary.bim +
hapmap10_auto_noofs-temporary.fam written.
138746 variants loaded from .bim file.
1184 people (589 males, 595 females) loaded from .fam.
196 people removed due to founder status (--filter-founders).
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354

## LD-prunning

In [13]:
!plink --file hapmap10_auto_noofs --indep-pairwise 50 10 0.1 --out keep
!plink --file hapmap10_auto_noofs --extract keep.prune.in --recode --out hapmap10_auto_noofs_ld

PLINK v1.90b4 64-bit (20 Mar 2017)             www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to keep.log.
Options in effect:
  --file hapmap10_auto_noofs
  --indep-pairwise 50 10 0.1
  --out keep

48283 MB RAM detected; reserving 24141 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (138746 variants, 988 people).
--file: keep-temporary.bed + keep-temporary.bim + keep-temporary.fam written.
138746 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.997722.
1387

## Different coding

In [14]:
!plink --file hapmap10_auto_noofs_ld --recode12 tab --out hapmap10_auto_noofs_ld_12
!plink --make-bed --file hapmap10_auto_noofs_ld --out hapmap10_auto_noofs_ld

PLINK v1.90b4 64-bit (20 Mar 2017)             www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Note: --recode12 flag deprecated.  Use 'recode 12 ...'.
Logging to hapmap10_auto_noofs_ld_12.log.
Options in effect:
  --file hapmap10_auto_noofs_ld
  --out hapmap10_auto_noofs_ld_12
  --recode 12 tab

48283 MB RAM detected; reserving 24141 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (55085 variants, 988 people).
--file: hapmap10_auto_noofs_ld_12-temporary.bed +
hapmap10_auto_noofs_ld_12-temporary.bim +
hapmap10_auto_noofs_ld_12-temporary.fam written.
55085 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546

## Single chromosome

In [15]:
!plink --recode --file hapmap10_auto_noofs --chr 2 --out hapmap10_auto_noofs_2

PLINK v1.90b4 64-bit (20 Mar 2017)             www.cog-genomics.org/plink/1.9/
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap10_auto_noofs_2.log.
Options in effect:
  --chr 2
  --file hapmap10_auto_noofs
  --out hapmap10_auto_noofs_2
  --recode

48283 MB RAM detected; reserving 24141 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (11647 variants, 988 people).
--file: hapmap10_auto_noofs_2-temporary.bed +
hapmap10_auto_noofs_2-temporary.bim + hapmap10_auto_noofs_2-temporary.fam
written.
11647 variants loaded from .bim file.
988 people (488 males, 500 females) loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 988 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384